<a href="https://colab.research.google.com/github/dj-1087/BigContest-COIN/blob/master/KerasTunerEX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam 

In [ ]:
#각층에 가지는 여러가지 하이퍼파라미터에 집중
inputs = Input(shape=(28,28,1))
x = Conv2D(32,(3,3),activation='relu')(inputs) #필터개수
x = Conv2D(32,(3,3),activation='relu')(x)
x = MaxPooling2D(strides=2)(x)
x = GlobalAveragePooling2D()(x) #최대풀링?평균풀링?
x = Dense(30, activation='softmax')(x)
x = Dropout(0.5)(x) #드롭아웃률
x = Dense(10,activation='softmax')(x)

model = Model(inputs = inputs, outputs=x)

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), #학습률 
              loss = 'categorical_crossentropy', 
              metrics = ['acc'])

**임의로 정한 하이퍼파라미터**


*   Conv2D층의 컨볼루션 필터 개수
*   Dense층의 은닉 유닛 개수
*   드롭아웃률
*   GAP or GMP의 사용
*   Adam 옵티마이저의 학습률



In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, Dense
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.optimizers import Adam

def build_model(hp):
    inputs = Input(shape=(28,28,1))
    x = inputs

    for i in range(hp.Int('n_layers',1,3)):
        #필터 개수 탐색
        x = Conv2D(filters=hp.Int('filters_'+str(i), 4, 64, step=8, default=16),
                   kernel_size = (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D(strides=2)(x)

    #GAP? GMP?
    if hp.Choice('global_pooling',['max','avg']) == 'avg':
        x = GlobalAveragePooling2D()(x)
    else:
        x = GlobalMaxPooling2D()(x)
    
    #은닉층의 개수 탐색
    x = Dense(units=hp.Int('units', min_value = 16, max_value = 128, step = 16),
              activation = 'relu')(x)
    
    #드롭아웃률 탐색
    x = Dropout(hp.Choice('dropout_rate', values = [0.2, 0.3, 0.5]))(x)
    x = Dense(10, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-3,1e-4,1e-5])),
                  loss = 'categorical_crossentropy',
                  metrics=['acc'])
    return model

**hp.Int(name, min_value, max_value, step=1, default=None)**


*   name: 인자의 이름을 지정, 반드시 모델 내에서 유일!
*   min_value: 탐색해야 할 값의 최소값을 지정
*   max_value: 탐색해야 할 값의 최대값을 지정
*   step: 단계를 지정
*   default: 기본값을 지정, 지정하지 않을 경우 min_value


**hp.Choice(name, values, default=None)**


*   name: 인자의 이름을 지정, 반드시 모델 내에서 유일!
*   values: 사용 가능한 값을 리스트 형태로 제공. 
            int, float, str, bool 사용가능
            리스트 안의 모든 값이 같은 타입이어야 함
*   default: 기본값 지정, 지정하지 않을 경우 None or 첫번째 항목


**hp.Fixed(name, value)**


*   name: 인자의 이름을 지정, 반드시 모델 내에서 유일!
*   value: 고정값 지정





In [ ]:
from tensorflow.keras.datasets import mnist

#텐서플로우 저장소에서 데이터 다운받기
(x_train, y_train), (x_test, y_test) = mnist.load_data(path='mnist.npz')

from sklearn.model_selection import train_test_split

#훈련/검증 데이터를 얻기 위해 0.7/0.3의 비율로 분리
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=777)
num_x_train = x_train.shape[0]
num_x_val = x_val.shape[0]
num_x_test = x_test.shape[0]

#모델의 입력으로 사용하기 위한 전처리 과정
x_train = (x_train.reshape(-1,28,28,1))/255
x_val = (x_val.reshape(-1,28,28,1))/255
x_test = (x_test.reshape(-1,28,28,1))/255

from tensorflow.keras.utils import to_categorical

#각 데이터의 레이블을 범주형 형태로 변경
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

11493376/11490434 [==============================] - 0s 0us/step


케라스 튜너의 탐색 방법에는 
**Hyperband, RandomSearch, Sklearn, Tuner** 클래스가 존재

In [ ]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(build_model,
                     objective='val_acc',   #모니터링 할 평가지표
                     max_trials=5,          #5개의 모델
                     executions_per_trial=3,#모델당 3번의 학습
                     directory='/content/gdrive/My Drive/Colab Notebooks/케라스튜너 공부',         #경로지정
                     project_name='helloworld')   #프로젝트명

In [ ]:
tuner.search_space_summary()

In [ ]:
tuner.search(x=x_train, y=y_train, epochs=10, validation_data=(x_val,y_val))

Epoch 1/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.1751 - acc: 0.1810 - val_loss: 1.9155 - val_acc: 0.3071
Epoch 2/10
1313/1313 [==============================] - 7s 5ms/step - loss: 1.7995 - acc: 0.3143 - val_loss: 1.6825 - val_acc: 0.3516
Epoch 3/10
1313/1313 [==============================] - 7s 5ms/step - loss: 1.6797 - acc: 0.3441 - val_loss: 1.6187 - val_acc: 0.3706
Epoch 4/10
1313/1313 [==============================] - 7s 5ms/step - loss: 1.6306 - acc: 0.3636 - val_loss: 1.5795 - val_acc: 0.3846
Epoch 5/10
1313/1313 [==============================] - 7s 5ms/step - loss: 1.5904 - acc: 0.3793 - val_loss: 1.5431 - val_acc: 0.4013
Epoch 6/10
1313/1313 [==============================] - 7s 5ms/step - loss: 1.5548 - acc: 0.3968 - val_loss: 1.5052 - val_acc: 0.4272
Epoch 7/10
1313/1313 [==============================] - 7s 5ms/step - loss: 1.5179 - acc: 0.4144 - val_loss: 1.4740 - val_acc: 0.4374
Epoch 8/10
1313/1313 [==============================] - 7s 5ms

Epoch 1/10
1313/1313 [==============================] - 7s 6ms/step - loss: 2.2962 - acc: 0.1501 - val_loss: 2.2851 - val_acc: 0.1242
Epoch 2/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2761 - acc: 0.2110 - val_loss: 2.2639 - val_acc: 0.3058
Epoch 3/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2514 - acc: 0.2512 - val_loss: 2.2324 - val_acc: 0.3493
Epoch 4/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2130 - acc: 0.2799 - val_loss: 2.1847 - val_acc: 0.4003
Epoch 5/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.1559 - acc: 0.3036 - val_loss: 2.1133 - val_acc: 0.4653
Epoch 6/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.0811 - acc: 0.3304 - val_loss: 2.0287 - val_acc: 0.5228
Epoch 7/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.0001 - acc: 0.3510 - val_loss: 1.9398 - val_acc: 0.5930
Epoch 8/10
1313/1313 [==============================] - 7s 5ms

Epoch 1/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.3015 - acc: 0.1051 - val_loss: 2.3005 - val_acc: 0.1115
Epoch 2/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2986 - acc: 0.1110 - val_loss: 2.2968 - val_acc: 0.1083
Epoch 3/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2946 - acc: 0.1128 - val_loss: 2.2924 - val_acc: 0.1100
Epoch 4/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2900 - acc: 0.1171 - val_loss: 2.2871 - val_acc: 0.1214
Epoch 5/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2842 - acc: 0.1295 - val_loss: 2.2808 - val_acc: 0.1424
Epoch 6/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2785 - acc: 0.1387 - val_loss: 2.2739 - val_acc: 0.1582
Epoch 7/10
1313/1313 [==============================] - 7s 5ms/step - loss: 2.2711 - acc: 0.1578 - val_loss: 2.2658 - val_acc: 0.1710
Epoch 8/10
1313/1313 [==============================] - 7s 5ms

Epoch 1/10
1313/1313 [==============================] - 6s 4ms/step - loss: 1.9299 - acc: 0.3063 - val_loss: 1.4960 - val_acc: 0.4866
Epoch 2/10
1313/1313 [==============================] - 6s 4ms/step - loss: 1.4109 - acc: 0.5031 - val_loss: 1.2962 - val_acc: 0.5609
Epoch 3/10
1313/1313 [==============================] - 6s 4ms/step - loss: 1.2739 - acc: 0.5588 - val_loss: 1.1912 - val_acc: 0.5963
Epoch 4/10
1313/1313 [==============================] - 6s 4ms/step - loss: 1.1997 - acc: 0.5844 - val_loss: 1.1268 - val_acc: 0.6143
Epoch 5/10
1313/1313 [==============================] - 6s 4ms/step - loss: 1.1548 - acc: 0.6023 - val_loss: 1.0997 - val_acc: 0.6211
Epoch 6/10
1313/1313 [==============================] - 5s 4ms/step - loss: 1.1208 - acc: 0.6132 - val_loss: 1.0610 - val_acc: 0.6354
Epoch 7/10
1313/1313 [==============================] - 5s 4ms/step - loss: 1.0941 - acc: 0.6243 - val_loss: 1.0403 - val_acc: 0.6419
Epoch 8/10
1313/1313 [==============================] - 5s 4ms

Epoch 1/10
1313/1313 [==============================] - 8s 6ms/step - loss: 2.2982 - acc: 0.1087 - val_loss: 2.2868 - val_acc: 0.0967
Epoch 2/10
1313/1313 [==============================] - 8s 6ms/step - loss: 2.2757 - acc: 0.1668 - val_loss: 2.2577 - val_acc: 0.3089
Epoch 3/10
1313/1313 [==============================] - 8s 6ms/step - loss: 2.2353 - acc: 0.2403 - val_loss: 2.1963 - val_acc: 0.4106
Epoch 4/10
1313/1313 [==============================] - 8s 6ms/step - loss: 2.1544 - acc: 0.2973 - val_loss: 2.0766 - val_acc: 0.4527
Epoch 5/10
1313/1313 [==============================] - 8s 6ms/step - loss: 2.0291 - acc: 0.3384 - val_loss: 1.9101 - val_acc: 0.5580
Epoch 6/10
1313/1313 [==============================] - 8s 6ms/step - loss: 1.8753 - acc: 0.3929 - val_loss: 1.7293 - val_acc: 0.6272
Epoch 7/10
1313/1313 [==============================] - 8s 6ms/step - loss: 1.7251 - acc: 0.4448 - val_loss: 1.5553 - val_acc: 0.6787
Epoch 8/10
1313/1313 [==============================] - 8s 6ms

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models()[0]
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 36)        360       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 28)        9100      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 52)        13156     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 52)        0         
_________________________________________________________________
global_max_pooling2d (Global (None, 52)                0         
_________________________________________________________________
dense (Dense)                (None, 96)               

In [ ]:
best_hp = tuner.get_best_hyperparameters()[0].values
best_hp

{'dropout_rate': 0.5,
 'filters_0': 36,
 'filters_1': 28,
 'filters_2': 52,
 'global_pooling': 'max',
 'learning_rate': 1e-05,
 'n_layers': 3,
 'units': 96}

In [ ]:
from kerastuner.tuners import  Hyperband
from kerastuner.applications import HyperResNet

hypermodel = HyperResNet(input_shape=(28,28,1),classes=10)

tuner = Hyperband(hypermodel,
                  objective = 'val_accuracy',
                  max_epochs = 10,
                  directory='/content/gdrive/My Drive/Colab Notebooks/케라스튜너 공부',         #경로지정
                  project_name='helloworld2'
                  )
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_val, y_val))

INFO:tensorflow:Reloading Oracle from existing project /content/gdrive/My Drive/Colab Notebooks/케라스튜너 공부/helloworld2/oracle.json
INFO:tensorflow:Reloading Tuner from /content/gdrive/My Drive/Colab Notebooks/케라스튜너 공부/helloworld2/tuner0.json
Epoch 1/2
1313/1313 [==============================] - 113s 86ms/step - loss: 2.4937 - accuracy: 0.4123 - val_loss: 1.4854 - val_accuracy: 0.7921
Epoch 2/2
1313/1313 [==============================] - 110s 84ms/step - loss: 0.8699 - accuracy: 0.7073 - val_loss: 9.4432 - val_accuracy: 0.8359


Epoch 1/2
1313/1313 [==============================] - 138s 105ms/step - loss: 3.0779 - accuracy: 0.1540 - val_loss: 482.1928 - val_accuracy: 0.1940
Epoch 2/2
1313/1313 [==============================] - 132s 100ms/step - loss: 1.4255 - accuracy: 0.4352 - val_loss: 4.7083 - val_accuracy: 0.1132


Epoch 1/2
1313/1313 [==============================] - 63s 48ms/step - loss: 0.4483 - accuracy: 0.8900 - val_loss: 1.2342 - val_accuracy: 0.7955
Epoch 2/2
1313/1313 [==============================] - 61s 47ms/step - loss: 0.2748 - accuracy: 0.9375 - val_loss: 2.8552 - val_accuracy: 0.5775


Epoch 1/2
1313/1313 [==============================] - 130s 99ms/step - loss: 3.5103 - accuracy: 0.1060 - val_loss: 6.7255 - val_accuracy: 0.1006
Epoch 2/2
1313/1313 [==============================] - 124s 95ms/step - loss: 2.3133 - accuracy: 0.1068 - val_loss: 6.2800 - val_accuracy: 0.0925


Epoch 1/2
1313/1313 [==============================] - 120s 92ms/step - loss: 4.5758 - accuracy: 0.3417 - val_loss: 1.6997 - val_accuracy: 0.4333
Epoch 2/2
1313/1313 [==============================] - 118s 90ms/step - loss: 1.0223 - accuracy: 0.8080 - val_loss: 1.6563 - val_accuracy: 0.6433


...

Epoch 1/10
1313/1313 [==============================] - 136s 103ms/step - loss: 1.9021 - accuracy: 0.3499 - val_loss: 1.9237 - val_accuracy: 0.4380
Epoch 2/10
1313/1313 [==============================] - 133s 101ms/step - loss: 0.5985 - accuracy: 0.8352 - val_loss: 1.8037 - val_accuracy: 0.6103
Epoch 3/10
1313/1313 [==============================] - 133s 102ms/step - loss: 0.3115 - accuracy: 0.9346 - val_loss: 2.3498 - val_accuracy: 0.8103
Epoch 4/10
1313/1313 [==============================] - 134s 102ms/step - loss: 0.3493 - accuracy: 0.9287 - val_loss: 0.7681 - val_accuracy: 0.9010
Epoch 5/10
1313/1313 [==============================] - 134s 102ms/step - loss: 0.3663 - accuracy: 0.9350 - val_loss: 0.1816 - val_accuracy: 0.9607
Epoch 6/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.2191 - accuracy: 0.9593 - val_loss: 0.3163 - val_accuracy: 0.9355
Epoch 7/10
1313/1313 [==============================] - 133s 101ms/step - loss: 0.2130 - accuracy: 0.9591 - val_l

Epoch 1/10
1313/1313 [==============================] - 136s 104ms/step - loss: 0.7671 - accuracy: 0.8205 - val_loss: 0.2717 - val_accuracy: 0.9314
Epoch 2/10
1313/1313 [==============================] - 134s 102ms/step - loss: 0.1658 - accuracy: 0.9533 - val_loss: 0.1225 - val_accuracy: 0.9647
Epoch 3/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.1011 - accuracy: 0.9713 - val_loss: 0.2141 - val_accuracy: 0.9433
Epoch 4/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.0974 - accuracy: 0.9734 - val_loss: 0.6665 - val_accuracy: 0.8414
Epoch 5/10
1313/1313 [==============================] - 134s 102ms/step - loss: 0.0865 - accuracy: 0.9753 - val_loss: 0.1036 - val_accuracy: 0.9728
Epoch 6/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.0783 - accuracy: 0.9777 - val_loss: 0.3732 - val_accuracy: 0.9069
Epoch 7/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.0717 - accuracy: 0.9802 - val_loss

Epoch 1/10
1313/1313 [==============================] - 131s 100ms/step - loss: 0.7845 - accuracy: 0.8273 - val_loss: 0.5857 - val_accuracy: 0.9319
Epoch 2/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.1788 - accuracy: 0.9618 - val_loss: 0.1514 - val_accuracy: 0.9632
Epoch 3/10
1313/1313 [==============================] - 128s 97ms/step - loss: 0.1209 - accuracy: 0.9760 - val_loss: 0.2013 - val_accuracy: 0.9474
Epoch 4/10
1313/1313 [==============================] - 129s 98ms/step - loss: 0.1412 - accuracy: 0.9809 - val_loss: 0.0985 - val_accuracy: 0.9748
Epoch 5/10
1313/1313 [==============================] - 128s 98ms/step - loss: 0.1035 - accuracy: 0.9837 - val_loss: 0.1069 - val_accuracy: 0.9746
Epoch 6/10
1313/1313 [==============================] - 127s 97ms/step - loss: 0.0946 - accuracy: 0.9855 - val_loss: 0.3418 - val_accuracy: 0.9556
Epoch 7/10
1313/1313 [==============================] - 127s 97ms/step - loss: 0.0792 - accuracy: 0.9867 - val_loss: 

Epoch 1/10
1313/1313 [==============================] - 135s 103ms/step - loss: 1.6702 - accuracy: 0.5769 - val_loss: 1.6620 - val_accuracy: 0.8273
Epoch 2/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.6503 - accuracy: 0.8305 - val_loss: 0.3073 - val_accuracy: 0.9061
Epoch 3/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.4889 - accuracy: 0.8774 - val_loss: 0.2688 - val_accuracy: 0.9278
Epoch 4/10
1313/1313 [==============================] - 130s 99ms/step - loss: 0.3389 - accuracy: 0.9138 - val_loss: 0.1754 - val_accuracy: 0.9493
Epoch 5/10
1313/1313 [==============================] - 129s 98ms/step - loss: 0.2788 - accuracy: 0.9308 - val_loss: 0.1815 - val_accuracy: 0.9468
Epoch 6/10
1313/1313 [==============================] - 129s 99ms/step - loss: 0.2131 - accuracy: 0.9433 - val_loss: 0.1291 - val_accuracy: 0.9622
Epoch 7/10
1313/1313 [==============================] - 129s 98ms/step - loss: 0.2029 - accuracy: 0.9453 - val_loss: 

INFO:tensorflow:Oracle triggered exit


In [ ]:
#HyperResNet: 옵티마이저, 손실함수, 평가지표 설정
tuner = Hyperband(hypermodel,
                  optimizer=tensorflow.keras.optimizers.Adam(1e-3),
                  loss='mse',
                  metrics=[])

In [ ]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 3.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=70ee4ec70026f447739d6b072bad768fdc70df448c99db2f74a6661c0b353c4d
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=cc9bf1bc8320be34b82afda13317ca74aa80c4c58564399fac747c2df67d6bf2
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables
